# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics  import mean_squared_error

In [2]:
df_0 = pd.read_csv('/datasets/geo_data_0.csv')
df_1 = pd.read_csv('/datasets/geo_data_1.csv')
df_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
df_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
df_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [5]:
df_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [6]:
print(df_0.info())
print(df_1.info())
print(df_2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column  

In [7]:
print(df_0.duplicated().sum())
print(df_1.duplicated().sum())
print(df_2.duplicated().sum())

0
0
0


Пропуски и дубликаты не обнаружены

In [8]:
df_0.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [9]:
df_1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [10]:
df_2.describe()


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


## Обучение и проверка модели

In [11]:
features_0 = df_0.drop(['product','id'],axis=1) 
target_0 = df_0['product']

In [12]:
features_0_train,features_0_valid,target_0_train,target_0_valid = train_test_split(features_0,target_0,
                                                                                   test_size = 0.25,
                                                                                   random_state = 12345)

In [13]:
print('Размер обучающей выборки',features_0_train.shape)
print('Размер валидационной выборки',features_0_valid.shape)

Размер обучающей выборки (75000, 3)
Размер валидационной выборки (25000, 3)


In [14]:
model = LinearRegression()
model.fit(features_0_train,target_0_train)
predictions_0_valid = model.predict(features_0_valid)
rmse_0_valid = mean_squared_error(target_0_valid,predictions_0_valid)**0.5
print('Средний запас  и rmse  предсказаного сырья на df_0 :',predictions_0_valid.mean(),rmse_0_valid)

Средний запас  и rmse  предсказаного сырья на df_0 : 92.59256778438035 37.5794217150813


In [15]:
features_1 = df_1.drop(['product','id'],axis=1) 
target_1 = df_1['product']

In [16]:
features_1_train,features_1_valid,target_1_train,target_1_valid = train_test_split(features_1,
                                                                                   target_1,
                                                                                   test_size = 0.25,
                                                                                   random_state = 12345)

In [17]:
print('Размер обучающей выборки',features_1_train.shape)
print('Размер валидационной выборки',features_1_valid.shape)

Размер обучающей выборки (75000, 3)
Размер валидационной выборки (25000, 3)


In [18]:
model = LinearRegression()
model.fit(features_1_train,target_1_train)
predictions_1_valid=model.predict(features_1_valid)
rmse_1_valid =mean_squared_error(target_1_valid,predictions_1_valid)**0.5
print('Средний запас  и rmse  предсказаного сырья на df_1 :',predictions_1_valid.mean(),rmse_1_valid)

Средний запас  и rmse  предсказаного сырья на df_1 : 68.728546895446 0.893099286775617


In [19]:
features_2 = df_2.drop(['product','id'],axis=1) 
target_2 = df_2['product']

In [20]:
features_2_train,features_2_valid,target_2_train,target_2_valid = train_test_split(features_2,
                                                                                   target_2,
                                                                                   test_size = 0.25,
                                                                                   random_state = 12345)

In [21]:
print('Размер обучающей выборки',features_2_train.shape)
print('Размер валидационной выборки',features_2_valid.shape)

Размер обучающей выборки (75000, 3)
Размер валидационной выборки (25000, 3)


In [22]:
model = LinearRegression()
model.fit(features_2_train,target_2_train)
predictions_2_valid=model.predict(features_2_valid)
rmse_2_valid =mean_squared_error(target_2_valid,predictions_2_valid)**0.5
print('Средний запас  и rmse  предсказаного сырья на df_2 :',predictions_2_valid.mean(),rmse_2_valid)

Средний запас  и rmse  предсказаного сырья на df_2 : 94.96504596800489 40.02970873393434


У первого и третьего региона,средний запас выше,чем у второго.  
У второго среднеквадратичная ошибка и расхождения между target и predictions минимальны.

## Подготовка к расчёту прибыли

In [23]:
money = 10000000000 #бюджет на разработку скважин в каждом регионе
price = 450000 #цена за 1000 баррелей
count = 200 #количество выбранных скважин на каждый регион
value = money/(price*count) # необходимый объем тысяч баррелей для безубыточности
print('Необходимый объем тысяч баррелей для безубыточности',value)

Необходимый объем тысяч баррелей для безубыточности 111.11111111111111


In [24]:
print('Средний объем в одной скважине в первом регионе',df_0['product'].mean())

Средний объем в одной скважине в первом регионе 92.50000000000001


In [25]:
print('Средний объем в одной скважине во втором регионе',df_1['product'].mean())

Средний объем в одной скважине во втором регионе 68.82500000000002


In [26]:
print('Средний объем в одной скважине в третьем регионе',df_2['product'].mean())

Средний объем в одной скважине в третьем регионе 95.00000000000004


Если судить по среднему для каждого региона,то в каждом регионе среднее меньше необходимого количества баррелей для безубыточности. Средний объем в одной скважине в регионе df_1 наименьший.

## Расчёт прибыли и рисков 

In [27]:
target_0_valid = target_0_valid.reset_index(drop=True) # разные индексы у target и predictions,использовала reset_index           
predictions_0_valid = pd.Series(predictions_0_valid,copy=True) # без reset_index,выдавало ошибку в последнем коде

target_1_valid = target_1_valid.reset_index(drop=True)
predictions_1_valid = pd.Series(predictions_1_valid,copy=True)

target_2_valid = target_2_valid.reset_index(drop=True)
predictions_2_valid = pd.Series(predictions_2_valid,copy=True)


In [28]:
def revenue (target,predictions,count):
    probs_sorted = predictions.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    return sum(selected)* price - money #объем запасов в скважине умножаем на стоимость и вычитаем расходы


In [29]:
profit_0 = revenue(target_0_valid,predictions_0_valid,200)
print('Прибыль первого региона',profit_0)

Прибыль первого региона 3320826043.1398544


In [30]:
profit_1 = revenue(target_1_valid,predictions_1_valid,200)
print('Прибыль второго региона',profit_1)

Прибыль второго региона 2415086696.681551


In [31]:
profit_2 = revenue(target_2_valid,predictions_2_valid,200)
print('Прибыль третьего региона',profit_2)

Прибыль третьего региона 2710349963.5998363


In [32]:
def boots (target, predictions):
    state = np.random.RandomState(12345) #вроде его не нужно использовать при бустрепе
    values = []
    for i in range(1000):
        target_subsample = target.sample(n=500, replace=True)
        preds_subsample = predictions[target_subsample.index]  #[target_subsample.index] выдает ошибку IndexError,может быть ошибка в этом
        values.append(revenue(target_subsample, preds_subsample,200)) #я очень долго пыталась устранить эту ошибку,и получилось толькоо так
        
    values = pd.Series(values)
    lower = values.quantile(0.025)
    higher = values.quantile(0.975)
    mean = values.mean()
    
    print('Средняя выручка',mean)
    print('Верхняя граница доверительного интервала', higher)
    print('Нижняя граница доверительного интервала', lower)
    print((values < 0).mean()* 100, "%")
    
print('Для первого региона')
boots(target_0_valid,predictions_0_valid)
print()
print('Для второго региона')
boots(target_1_valid,predictions_1_valid)
print()
print('Для третьего региона')
boots(target_2_valid,predictions_2_valid)

Для первого региона
Средняя выручка 417853525.1880401
Верхняя граница доверительного интервала 940272846.9630082
Нижняя граница доверительного интервала -130225922.88137779
7.1 %

Для второго региона
Средняя выручка 497659655.356162
Верхняя граница доверительного интервала 941006630.8833152
Нижняя граница доверительного интервала 65362044.78921509
1.0 %

Для третьего региона
Средняя выручка 421528654.9720042
Верхняя граница доверительного интервала 930427214.7262981
Нижняя граница доверительного интервала -146596524.62841755
6.800000000000001 %


## Общий вывод

Регион df_1 больше всего подходит для разработки скважин.
Вероятность получить убытки меньше 2,5% ,имеет регион df_1,так же у региона самый низкий процент вероятности получить убытки, по сравнению с другими регионами. Регион df_1 имеет наибольшую среднюю выручку,по сравнению с остальными регионами.


## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован